# (연구&보람) 신용카드거래 사기탐지 – 데이터(8, df02)커널죽음

김보람  
2023-08-25

In [12]:
import numpy as np
import pandas as pd
from sklearn import model_selection # split함수이용

`-` fraudTrain

In [2]:
fraudTrain = pd.read_csv("fraudTrain.csv").iloc[:,1:]

In [3]:
fraudTrain = fraudTrain.assign(trans_date_trans_time= list(map(lambda x: pd.to_datetime(x), fraudTrain.trans_date_trans_time)))
fraudTrain

1048575 rows × 22 columns

`-` df02

In [19]:
_df1 = fraudTrain[fraudTrain["is_fraud"] == 0].sample(frac=0.20, random_state=42)
_df2 = fraudTrain[fraudTrain["is_fraud"] == 1]
df02 = pd.concat([_df1,_df2])
df02 = df02.reset_index()

`-` df_toy

In [44]:
df_toy=df02[:5].copy()
df_toy.cc_num = pd.Series([1,1,1,2,2])
df_toy

5 rows × 23 columns

`-` df_toy 에서 time_difference 구함

**고객1**

In [41]:
df_toy.iloc[0].trans_date_trans_time.value - df_toy.iloc[1].trans_date_trans_time.value

22914900000000000

In [45]:
df_toy.iloc[0].trans_date_trans_time.value - df_toy.iloc[2].trans_date_trans_time.value

17453520000000000

In [48]:
df_toy.iloc[1].trans_date_trans_time.value - df_toy.iloc[2].trans_date_trans_time.value

-5461380000000000

**고객2**

In [46]:
df_toy.iloc[3].trans_date_trans_time.value - df_toy.iloc[4].trans_date_trans_time.value

30729180000000000

**고객1,2**

In [52]:
def compute_time_difference(group):
    n = len(group)
    result = []
    for i in range(n):
        for j in range(n):
            time_difference = abs(group.iloc[i].trans_date_trans_time.value - group.iloc[j].trans_date_trans_time.value)
            result.append([group.iloc[i].name, group.iloc[j].name, time_difference])
    return result

In [53]:
groups = df_toy.groupby('cc_num')
edge_index_list_plus = [compute_time_difference(group) for _, group in groups]
edge_index_list_plus_flat = [item for sublist in edge_index_list_plus for item in sublist]
edge_index_list_plus_nparr = np.array(edge_index_list_plus_flat)
edge_index_list_plus_nparr

array([[                0,                 0,                 0],
       [                0,                 1, 22914900000000000],
       [                0,                 2, 17453520000000000],
       [                1,                 0, 22914900000000000],
       [                1,                 1,                 0],
       [                1,                 2,  5461380000000000],
       [                2,                 0, 17453520000000000],
       [                2,                 1,  5461380000000000],
       [                2,                 2,                 0],
       [                3,                 3,                 0],
       [                3,                 4, 30729180000000000],
       [                4,                 3, 30729180000000000],
       [                4,                 4,                 0]])

`-` df02에서 time_difference 구함

In [46]:
t1 = time.time()
groups = df02.groupby('cc_num')
edge_index_list_plus = [compute_time_difference(group) for _, group in groups]
edge_index_list_plus_flat = [item for sublist in edge_index_list_plus for item in sublist]
edge_index_list_plus_nparr = np.array(edge_index_list_plus_flat)
np.save('edge_index_list_plus02.npy', edge_index_list_plus_nparr)
t2 = time.time()
t2-t1

15537.573440551758

In [48]:
(t2-t1)/3600

4.3159926223754885